In [47]:
import psycopg2
from password import password

In [48]:
password = password

In [49]:
def connect_and_make_cursor(host, dbname, user, password):
    """host = IP of host service
    dbname = database which to connect to
    user = username
    password = password associated with username"""
    
    try:
        conn = psycopg2.connect(f"host = {host}, dbname = {dbname} user = {user} password = {password}")
        conn.set_session(autocommit = True)
        cur = conn.cursor()
    except psycopg2.Error as e:
        print(e)
    return conn, cur

In [50]:
conn, cur = connect_and_make_cursor('127.0.0.1', 'postgres', 'postgres', password)

# CREATE DATABASE

In [51]:
cur.execute("CREATE DATABASE acme_co")

In [52]:
def close_connection():
    """closes both cursor and connection in that order"""
    
    cur.close()
    conn.close()

In [53]:
close_connection()

In [54]:
conn, cur = connect_and_make_cursor('127.0.0.1', 'acme_co', 'postgres', password)

# CREATE TABLES

In [55]:
def create_table(table_name):
    """table_name = name of the table which you like to create"""
    try:
        cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ();")
    except psycopg2.Error as e:
        print(e)

In [56]:
tables = ['employee', 'sales_dept', 'it_dept', 'customers', 'inventory', 'sales']

for table in tables:
    create_table(table)

In [57]:
employee_cols = {'emp_id' : 'SERIAL PRIMARY KEY', 'first_name': 'varchar', 'last_name': 'varchar', 'position': 'varchar', 'start_date': 'DATE DEFAULT CURRENT_DATE', 'phone': 'varchar', 'email': 'varchar'}

In [58]:
def insert_columns(table_name, col_type_dict):
    """ table_name = Name of Table to insert columns
        col_type_dict = Dictionary containing key value pairs of column name : column type"""
    try:
        for col_name, type_ in col_type_dict.items():
            cur.execute(f"ALTER TABLE {table_name} \
                        ADD COLUMN IF NOT EXISTS {col_name} {type_};")
    except psycopg2.Error as e:
        print(e)

In [59]:
insert_columns('employee', employee_cols)

In [60]:
def get_column_names(table_name):
    """returns column names of a specific table"""
    
    try:
        cur.execute(f"select column_name, data_type, character_maximum_length \
        from INFORMATION_SCHEMA.COLUMNS where table_name = '{table_name}';")
    except psycopg2.Error as e:
        print(e)
        
    return cur.fetchall()

In [61]:
get_column_names('employee')

[('emp_id', 'integer', None),
 ('first_name', 'character varying', None),
 ('last_name', 'character varying', None),
 ('position', 'character varying', None),
 ('start_date', 'date', None),
 ('phone', 'character varying', None),
 ('email', 'character varying', None)]

In [62]:
def insert_rows(table_name, cols, data):
    """table_name = table which to insert rows into
    cols = list of columns which will recieve the data. Expects string in format "(col1, col2, col3, coln)"
    data = the actual data which you will like to insert into each row contained in tuple"""
    
    count = len(data)
    query = f"INSERT INTO {table_name} {cols} \
                    VALUES (" + "%s,"*count  + ")"
    query = query[::-1].replace(',', '', 1)
    query = query[::-1]
    try:
        cur.execute(query, data)
    except psycopg2.Error as e:
        print(e)

In [63]:
employee_list = [('Bob', 'Smith', 'President', '510-123-4567', 'bob_smith@acme.com'),
                ('Jane', 'Smith', 'Vice President', '510-123-4537', 'jane_smith@acme.com'),
                ('Tom', 'Harper', 'Developer', '510-123-4577', 'tom_h@acme.com'),
                ('Vina', 'Nguyen', 'Sales', '510-123-4517', 'vina_n@acme.com'),
                ('Lee', 'Garr', 'Sales', '510-123-4597', 'lee_g@acme.com')]

for employee in employee_list:
    insert_rows('employee', "(first_name, last_name, position, phone, email)", employee)

In [64]:
cur.execute("SELECT * FROM employee")
cur.fetchall()

[(1,
  'Bob',
  'Smith',
  'President',
  datetime.date(2019, 3, 31),
  '510-123-4567',
  'bob_smith@acme.com'),
 (2,
  'Jane',
  'Smith',
  'Vice President',
  datetime.date(2019, 3, 31),
  '510-123-4537',
  'jane_smith@acme.com'),
 (3,
  'Tom',
  'Harper',
  'Developer',
  datetime.date(2019, 3, 31),
  '510-123-4577',
  'tom_h@acme.com'),
 (4,
  'Vina',
  'Nguyen',
  'Sales',
  datetime.date(2019, 3, 31),
  '510-123-4517',
  'vina_n@acme.com'),
 (5,
  'Lee',
  'Garr',
  'Sales',
  datetime.date(2019, 3, 31),
  '510-123-4597',
  'lee_g@acme.com')]

# Whoops. The start_date is wrong for our President and Vice President. Let's update these values

In [65]:
def update_row(table_name, col_name, data, condition = None):
    """table_name = table which to update
    col_name = column in which to update the value
    data = data which you would like to change the field to
    condition = Optional. Conditional requirement with which to query using WHERE clause"""
    
    if condition != None:
        try:
            cur.execute(f"UPDATE {table_name}\
            SET {col_name} = {data} \
            WHERE {condition};")
        except psycopg2.Error as e:
            print(e)
            
    else:
        try:
            cur.execute(f"UPDATE {table_name}\
            SET {col_name} = {data};")
        except psycopg2.Error as e:
            print(e)
        

In [66]:
update_row('employee', 'start_date', "'2019/01/23'", "LAST_NAME = 'Smith'")

In [67]:
cur.execute("SELECT * FROM employee")
cur.fetchall()

[(3,
  'Tom',
  'Harper',
  'Developer',
  datetime.date(2019, 3, 31),
  '510-123-4577',
  'tom_h@acme.com'),
 (4,
  'Vina',
  'Nguyen',
  'Sales',
  datetime.date(2019, 3, 31),
  '510-123-4517',
  'vina_n@acme.com'),
 (5,
  'Lee',
  'Garr',
  'Sales',
  datetime.date(2019, 3, 31),
  '510-123-4597',
  'lee_g@acme.com'),
 (1,
  'Bob',
  'Smith',
  'President',
  datetime.date(2019, 1, 23),
  '510-123-4567',
  'bob_smith@acme.com'),
 (2,
  'Jane',
  'Smith',
  'Vice President',
  datetime.date(2019, 1, 23),
  '510-123-4537',
  'jane_smith@acme.com')]

# Extract sales staff data in order to create our sales_dept table

In [68]:
cur.execute("SELECT emp_id FROM employee \
WHERE position = 'Sales'")

sales_staff = cur.fetchall()
sales_staff

[(4,), (5,)]

In [69]:
sales_cols = {'emp_id' : 'int PRIMARY KEY', 'total_sales': 'int', 'region': 'varchar'}

In [70]:
insert_columns('sales_dept', sales_cols)

In [71]:
get_column_names('sales_dept')

[('emp_id', 'integer', None),
 ('total_sales', 'integer', None),
 ('region', 'character varying', None)]

In [72]:
sales_list = [(4, 10000, 'NW'), (5, 5000, 'SW')]

for sales_rep in sales_list:
    insert_rows('sales_dept', "(emp_id, total_sales, region)", sales_rep)

In [73]:
cur.execute("SELECT * FROM sales_dept")
cur.fetchall()

[(4, 10000, 'NW'), (5, 5000, 'SW')]

** Fetch employee name and total sales using inner join on employee and sales department **

In [74]:
cur.execute("SELECT EMPLOYEE.first_name, EMPLOYEE.last_name, SALES_DEPT.total_sales \
FROM employee \
INNER JOIN sales_dept ON sales_dept.emp_id = employee.emp_id")

In [75]:
cur.fetchall()

[('Vina', 'Nguyen', 10000), ('Lee', 'Garr', 5000)]

# Create IT dept table

In [76]:
get_column_names('it_dept')

[]

In [77]:
cur.execute("SELECT * FROM employee WHERE position = 'Developer'")
cur.fetchall()

[(3,
  'Tom',
  'Harper',
  'Developer',
  datetime.date(2019, 3, 31),
  '510-123-4577',
  'tom_h@acme.com')]

In [78]:
it_cols = {'emp_id' : 'int PRIMARY KEY', 'skills': 'text[]', 'remote': 'bool'}

insert_columns('it_dept', it_cols)

In [79]:
get_column_names('it_dept')

[('emp_id', 'integer', None),
 ('skills', 'ARRAY', None),
 ('remote', 'boolean', None)]

In [80]:
insert_rows('it_dept', "(emp_id, skills, remote)", (3, ['Python', 'SQL', 'Javascript'], True))

In [81]:
cur.execute("SELECT * FROM it_dept")
cur.fetchall()

[(3, ['Python', 'SQL', 'Javascript'], True)]

** Query by individual programming skill set **

In [82]:
cur.execute("SELECT * FROM it_dept WHERE 'Python' = ANY(skills)")
cur.fetchall()

[(3, ['Python', 'SQL', 'Javascript'], True)]

# Create customers Table

In [83]:
cust_columns = {"cust_id": "SERIAL PRIMARY KEY", "first_name": 'varchar', 'last_name': 'varchar', 'sales_man': 'int references sales_dept(emp_id)', 'contact': 'varchar'}

insert_columns('customers', cust_columns)

In [84]:
customers = [('janet', 'dane', 4, 'janet_dane@gmail.com'), ('Joey', 'Capone', 5, 'joey_capone@gmail.com')]

In [85]:
for customer in customers:
    insert_rows('customers', "(first_name, last_name, sales_man, contact)", customer)

In [86]:
cur.execute("SELECT * FROM customers")
cur.fetchall()

[(1, 'janet', 'dane', 4, 'janet_dane@gmail.com'),
 (2, 'Joey', 'Capone', 5, 'joey_capone@gmail.com')]

# CREATE SALES TABLE

In [87]:
sales_cols = {'order_id' : 'SERIAL PRIMARY KEY', 'cust_id': 'int references customers(cust_id)', 'purchase_total': 'int',
             'total_items': 'int', 'salesman_id': 'int references sales_dept(emp_id)', 'description': "text[]" }

In [88]:
insert_columns('sales', sales_cols)

In [89]:
dummy_sales = [(1, 600, 2, 4, ['GPU', 'monitor']), (2, 1200, 3, 5, ['GPU', 'monitor', 'CPU'])]

for sale in dummy_sales:
    insert_rows('sales', "(cust_id, purchase_total, total_items, salesman_id, description)", sale)

In [92]:
get_column_names('sales')

[('order_id', 'integer', None),
 ('cust_id', 'integer', None),
 ('purchase_total', 'integer', None),
 ('total_items', 'integer', None),
 ('salesman_id', 'integer', None),
 ('description', 'ARRAY', None)]

In [91]:
cur.execute("SELECT * FROM sales")
cur.fetchall()

[(1, 1, 600, 2, 4, ['GPU', 'monitor']),
 (2, 2, 1200, 3, 5, ['GPU', 'monitor', 'CPU'])]

** Let's query the average purchase total and round to two decimal places**

In [103]:
cur.execute("SELECT ROUND(AVG(purchase_total), 2) FROM sales;")
cur.fetchall()

[(Decimal('900.00'),)]

** Now we will query all purchase descriptions sold, salesman first and last name associated with purchase and the region from which they work. This will require 2 joins.**

In [105]:
cur.execute("SELECT sales.description, sales.total_items, sales_dept.region, \
employee.first_name, employee.last_name FROM SALES \
INNER JOIN sales_dept ON sales.salesman_id = sales_dept.emp_id \
INNER JOIN employee ON sales_dept.emp_id = employee.emp_id")
cur.fetchall()

[(['GPU', 'monitor'], 2, 'NW', 'Vina', 'Nguyen'),
 (['GPU', 'monitor', 'CPU'], 3, 'SW', 'Lee', 'Garr')]

# Create Inventory Table

In [106]:
inventory_cols = {"item_id": "SERIAL PRIMARY KEY", "item_name": "varchar", "in_stock": "bool", "total": "int"}

In [108]:
insert_columns("inventory", inventory_cols)

In [109]:
get_column_names('inventory')

[('item_id', 'integer', None),
 ('item_name', 'character varying', None),
 ('in_stock', 'boolean', None),
 ('total', 'integer', None)]

** Whoops forgot to add our wholesale price and retail price columns in the inventory. Luckily our function can easily update the table **

In [110]:
insert_columns("inventory", {"wholesale_price": "int", "retail_price": "int"})

In [113]:
get_column_names('inventory')

[i[0] for i in get_column_names('inventory')]

['item_id',
 'item_name',
 'in_stock',
 'total',
 'wholesale_price',
 'retail_price']

In [116]:
dummy_inventory = [('GPU', True, 100, 260, 320), ('CPU', True, 120, 180, 260), ('monitor', True, 88, 120, 160),
                  ('Mouse', False, 0, 30, 45)]

for item in dummy_inventory:
    insert_rows("inventory", "(item_name, in_stock, total, wholesale_price, retail_price)", item)

In [117]:
cur.execute("SELECT * FROM inventory")
cur.fetchall()

[(1, 'GPU', True, 100, 260, 320),
 (2, 'CPU', True, 120, 180, 260),
 (3, 'monitor', True, 88, 120, 160),
 (4, 'Mouse', False, 0, 30, 45)]

In [ ]:
close_connection()